# Trabalho Prático 1 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## Exercício 1

In [4]:
'''
Imports necessários para a execução deste mesmo Notebook
'''
import os

O principal objetivo deste primeiro exercício prático passava pela implementação de uma **comunicação síncrona** e **segura** entre duas entidades, um **receiver** e um **emitter** seguindo determinados parâmetros que vão sendo explicitados e demonstrados em cada uma das alíneas que se seguem.

### Alínea a - Implementar um gerador de nounces

Como se encontra explícito no próprio título desta subsecção, a principal finalidade desta alínea era a implementação de um **gerador de nounces** de modo a que, sempre que seja solicitado uma nova instância de **nounce**, temos a garantia de que ele é **aleatório** e ainda que **nunca foi retornado**, numa ocasião prévia por esse mesmo gerador.

Com isto em mente, desenvolvemos a seguinte classe em **Python**, a qual apenas apresenta um método que permite o acesso a um novo **nounce**, designado de **get()**:

In [5]:
''' 
Classe que permite gerar 
nounces que nunca se repetem
'''
class NGenerator :

    '''
    Construtor para objetos da classe NGenerator
    '''
    def __init__(self) :
        self.historic = []

    ''' 
    Método que nos permite obter um novo nounce
    '''
    def get(self) :
        nounce = os.urandom(12)
        while nounce in self.historic :
            nounce = os.urandom(12)
        self.historic.append(nounce)
        return nounce

> De seguida, mostramos o exemplo de execução do método que permite obter uma nova instância de **nounce**.

In [9]:
ng = NGenerator()
ng.get()

b'\x04\xbcg\xf9p\xba\x9d\x18\xb0\xcc\x0e\xd6'